# Summary metric datasets
- Merged datasets will be grouped by `building_id` and `date` to calculate RMSLE. One dataset per meter, considering all 50 submissions.

In [2]:
import pandas as pd
import numpy as np
import glob
import gzip
import pickle
from datetime import datetime
from sklearn.metrics import mean_squared_log_error

import sys
sys.path.append("..\\source\\")
import utils as utils

In [3]:
path_in = "..\\data\\processed\\merged\\"
path_out = "..\\data\\processed\\summary\\"

In [17]:
def RMSLE(y_real, y_pred):
    rmsle = np.sqrt(mean_squared_log_error( y_real, y_pred ))
    return rmsle

## Create dataset by meter

In [9]:
files = glob.glob(path_in + '*')
len(files)

50

In [10]:
df = pd.read_pickle(files[1])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   submission   float32
 1   res          float32
 2   building_id  int16  
 3   meter        int8   
 4   timestamp    object 
dtypes: float32(2), int16(1), int8(1), object(1)
memory usage: 1.0+ GB


In [11]:
df.head()

,submission,res,building_id,meter,timestamp
0,173.370300,173.369995,0,0,2017-01-01 00:00:00
1,53.512718,53.512699,1,0,2017-01-01 00:00:00
2,6.143042,6.143000,2,0,2017-01-01 00:00:00
3,101.701469,101.700996,3,0,2017-01-01 00:00:00
4,1141.240723,1141.239990,4,0,2017-01-01 00:00:00


In [12]:
# Meter
meter = 0
metername = {0: "electricity", 1: "chilledwater", 2: "steam", 3: "hotwater"}

# Files to convert
start = 0
end = 2

dfs = []
for datafile in files[start:end]:

    # file id
    name = int(datafile.split("\\")[-1].split("_")[0].split("sub")[1])

    # Print progress
    number = files.index(datafile) + 1
    total = len(files)
    progress = round(number * 100 / total,2)
    print(f"{name} - {progress}% ({number} of {len(files)})")

    # Load data
    df = pd.read_pickle(datafile)
    # rename
    try:
        df = df.rename(columns={"res":"meter_reading"})
    except:
        pass
    print(f"Dataset {number} loaded")

    # Filter data
    df = df[df.meter == meter]
    print(f"Dataset {number} filtered")

    # Replace and drop all negative values
    df.loc[df.submission < 0, "submission"] = np.nan
    df.loc[df.meter_reading < 0, "meter_reading"] = np.nan
    df.dropna(inplace=True)
    # Convert to timestamp
    df.timestamp = pd.to_datetime(df.timestamp, format="%Y-%m-%d %H:%M:%S")
    # Add submission id
    df["sub_id"] = name
    print(f"Dataset {number} transformed")

    # Append to list
    dfs.append(df)
    print(f"Dataset {number} append")

# Concat all
meter_df = pd.concat(dfs)  

# Export df
file = gzip.GzipFile(path_out + f'by_meter\\{metername[meter]}.pickle.gz', 'wb', 6)
file.write(pickle.dumps(meter_df))
file.close()
print(f"Data exported")

13577404 - 2.0% (1 of 50)
Dataset 1 loaded
Dataset 1 filtered
Dataset 1 transformed
Dataset 1 append
13768618 - 4.0% (2 of 50)
Dataset 2 loaded
Dataset 2 filtered
Dataset 2 transformed
Dataset 2 append
Data exported


## Calculate RMSLE

In [14]:
df = pd.read_pickle(path_out + f"by_meter\\electricity.pickle.gz")
df.timestamp = pd.to_datetime(df.timestamp, format="%Y-%m-%d %H:%M:%S")

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48325390 entries, 0 to 41693189
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   submission     float32       
 1   meter_reading  float32       
 2   building_id    int16         
 3   meter          int8          
 4   timestamp      datetime64[ns]
 5   sub_id         int64         
dtypes: datetime64[ns](1), float32(2), int16(1), int64(1), int8(1)
memory usage: 2.2 GB


In [15]:
df.head(20)

,submission,meter_reading,building_id,meter,timestamp,sub_id
0,173.370300,173.369995,0,0,2017-01-01,13577404
1,53.512699,53.512699,1,0,2017-01-01,13577404
2,6.143000,6.143000,2,0,2017-01-01,13577404
3,101.701500,101.700996,3,0,2017-01-01,13577404
4,1141.240723,1141.239990,4,0,2017-01-01,13577404
5,13.651200,13.651200,5,0,2017-01-01,13577404
6,260.874512,260.875000,6,0,2017-01-01,13577404
7,502.705597,502.705994,7,0,2017-01-01,13577404
9,513.285278,513.284973,8,0,2017-01-01,13577404
10,61.840000,61.840000,9,0,2017-01-01,13577404


In [18]:
# Meter
metername = {0: "electricity", 1: "chilledwater", 2: "steam", 3: "hotwater"}
meter_list = list(metername.keys())

for meter in meter_list:

    # Load data
    df = pd.read_pickle(path_out + f"by_meter\\{metername[meter]}.pickle.gz")
    print(f"Data loaded")

    # Get summary metrics
    print("Grouping by bdg, date")

    # Group
    df = pd.DataFrame(df.groupby(["building_id",df.timestamp.dt.date])["submission","meter_reading"].apply(lambda group: RMSLE(group["meter_reading"], group["submission"]))).reset_index()

    # Rename columns
    df = df.rename(columns={0:"rmsle"})
    print("Data summary created")

    # Export df
    file = gzip.GzipFile(path_out + f'summary\\{metername[meter]}_RMSLE.pickle.gz', 'wb', 6)
    file.write(pickle.dumps(df))
    file.close()
    print(f"Data summary exported")

Data loaded
Grouping by bdg, date
Data summary created
Data summary exported


In [22]:
len(df.timestamp.unique())

730